МЛ практика 6. Обучение без учителя

**Задание состоит из следующих частей:**

    Определение обучения без учителя
    Разбор алгоритмов DBSSCAN, K-means
    Метрики качества кластеризации
    Пример полезных задач, которые решает кластеризация

P.S. Некоторые вещи (например PCA, обработка текстовых данных) мы пройдем чуть позже.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Загрузка набора данных
path_to_file: str = 'content/income.csv'
df = pd.read_csv(path_to_file)

In [2]:
# Создаем копию данных
category_columns: list[str] = df.select_dtypes(include=['object']).columns # собираем колонки помеченные как objects
df.info()
print(category_columns)


# Применяем One-Hot Encoding
df = pd.get_dummies(df, columns=category_columns, drop_first=True)
print(df.columns)
df.education_9th = df.education_9th.astype(int)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       30725 non-null  object
 2   education       32561 non-null  object
 3   education-num   32561 non-null  int64 
 4   marital-status  32561 non-null  object
 5   occupation      30718 non-null  object
 6   relationship    32561 non-null  object
 7   race            32561 non-null  object
 8   sex             32561 non-null  object
 9   capital-gain    32561 non-null  int64 
 10  capital-loss    32561 non-null  int64 
 11  hours-per-week  32561 non-null  int64 
 12  native-country  31978 non-null  object
 13  income >50K     32561 non-null  int64 
dtypes: int64(6), object(8)
memory usage: 3.5+ MB
Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='ob

In [3]:

from sklearn.preprocessing import MinMaxScaler
# Выбираем числовые признаки
numeric_features = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Инициализируем scaler
scaler = MinMaxScaler()

# Применяем нормализацию
df[numeric_features] = scaler.fit_transform(df[numeric_features])

In [4]:
df

age  education-num  capital-gain  capital-loss  hours-per-week  \
0      0.301370       0.800000      0.021740           0.0        0.397959   
1      0.452055       0.800000      0.000000           0.0        0.122449   
2      0.287671       0.533333      0.000000           0.0        0.397959   
3      0.493151       0.400000      0.000000           0.0        0.397959   
4      0.150685       0.800000      0.000000           0.0        0.397959   
...         ...            ...           ...           ...             ...   
32556  0.136986       0.733333      0.000000           0.0        0.377551   
32557  0.315068       0.533333      0.000000           0.0        0.397959   
32558  0.561644       0.533333      0.000000           0.0        0.397959   
32559  0.068493       0.533333      0.000000           0.0        0.193878   
32560  0.479452       0.533333      0.150242           0.0        0.397959   

       income >50K  workclass_Local-gov  workclass_Never-worked  \
0              0.0                False                   False   
1              0.0                False                   False   
2              0.0                False                   False   
3              0.0                False                   False   
4              0.0                False                   False   
...            ...                  ...                     ...   
32556          0.0                False                   False   
32557          1.0                False                   False   
32558          0.0                False                   False   
32559          0.0                False                   False   
32560          1.0                False                   False   

       workclass_Private  workclass_Self-emp-inc  ...  \
0                  False                   False  ...   
1                  False                   False  ...   
2                   True                   False  ...   
3                   True                   False  ...   
4                   True                   False  ...   
...                  ...                     ...  ...   
32556               True                   False  ...   
32557               True                   False  ...   
32558               True                   False  ...   
32559               True                   False  ...   
32560              False                    True  ...   

       native-country_Portugal  native-country_Puerto-Rico  \
0                        False                       False   
1                        False                       False   
2                        False                       False   
3                        False                       False   
4                        False                       False   
...                        ...                         ...   
32556                    False                       False   
32557                    False                       False   
32558                    False                       False   
32559                    False                       False   
32560                    False                       False   

       native-country_Scotland  native-country_South  native-country_Taiwan  \
0                        False                 False                  False   
1                        False                 False                  False   
2                        False                 False                  False   
3                        False                 False                  False   
4                        False                 False                  False   
...                        ...                   ...                    ...   
32556                    False                 False                  False   
32557                    False                 False                  False   
32558                    False                 False                  False   
32559                    False                 Fal


О кластеризации и задачи обучения без учителя
Обучение с учителем

Представьте, что у вас есть список задач с правильными ответами. Например, вам дали примеры: «Это яблоко», «Это груша».

Задача — научиться отличать их, чтобы потом, когда вы увидите новый фрукт, вы смогли правильно его назвать. То есть у вас есть учитель — тот, кто заранее всё объяснил и дал правильные ответы.
Обучение без учителя

А теперь представьте, что вам дали корзину фруктов, но никто не сказал, как они называются.

Задача — посмотреть на фрукты и найти, что у них общего, например: круглые положить в одну группу, длинные в другую. Тут вы работаете без подсказок и сами решаете, как группировать.
Разница

    В задачах с учителем есть правильные ответы, и мы обучем алгоритм делать предсказания на основе примеров.
    В задачах без учителя правильных ответов нет, ищем закономерности и пытаемся найти смысл группировки данные самостоятельно.

Пример:

    С учителем: Мы ищем лучшее вино.
    Без учителя: Мы пытаемся объеденить вина, чтобы понять по какому принципу они объеденяются.

K-means

K-means — это алгоритм кластеризации, который делит данные на 𝑘 групп (кластеров). Основная идея: найти центры кластеров так, чтобы точки внутри одного кластера были ближе друг к другу, чем к точкам из других кластеров.

    Инициализация: выбираем 𝑘 случайных центров (центроидов).
    Присвоение точек: каждая точка относится к ближайшему центру. Обновление центров: пересчитываем центры кластеров как среднее всех точек в кластере.
    Повторяем шаги 2–3, пока центры не перестанут меняться или не достигнем максимального числа итераций.



In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

def interactive_kmeans_visualization(X, max_clusters=10, max_iterations=50):

    # преобразуем X в numpy array
    if isinstance(X, pd.DataFrame):
        X = X.values

    def plot_kmeans_iteration(X, centroids, labels=None, step=0):
        clear_output(wait=True)
        plt.figure(figsize=(8, 6))
        if labels is not None:
            plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', s=30, alpha=0.6)
        else:
            plt.scatter(X[:, 0], X[:, 1], s=30, alpha=0.6)
        plt.scatter(centroids[:, 0], centroids[:, 1], c='red', marker='x', s=100, label='Centroids')
        plt.title(f'K-means Iteration {step}')
        plt.legend()
        plt.show()

    def k_means_interactive(X, k, max_iters):
        np.random.seed(42)
        centroids = X[np.random.choice(X.shape[0], k, replace=False)]
        for step in range(max_iters):
            distances = np.linalg.norm(X[:, np.newaxis] - centroids, axis=2)
            labels = np.argmin(distances, axis=1)
            plot_kmeans_iteration(X, centroids, labels, step)
            new_centroids = np.array([X[labels == i].mean(axis=0) for i in range(k)])
            if np.all(centroids == new_centroids):
                break
            centroids = new_centroids

    def run_kmeans(k, max_iters):
        k_means_interactive(X, k, max_iters)

    k_slider = widgets.IntSlider(value=3, min=2, max=max_clusters, step=1, description='Clusters (k):')
    iter_slider = widgets.IntSlider(value=10, min=1, max=max_iterations, step=1, description='Max Iterations:')

    interactive_plot = widgets.interactive(run_kmeans, k=k_slider, max_iters=iter_slider)
    display(interactive_plot)

# Берем датасет и визуализируем K-means
interactive_kmeans_visualization(df[:100])

interactive(children=(IntSlider(value=3, description='Clusters (k):', max=10, min=2), IntSlider(value=10, desc…

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Определяем количество кластеров k
k = 3  # Вы можете изменить это значение в соответствии с вашими данными

# Создаем и обучаем модель KMeans
kmeans = KMeans(n_clusters=k, random_state=42)
labels = kmeans.fit_predict(df)
centroids = kmeans.cluster_centers_

# Добавляем метки кластеров в исходный DataFrame
df_k_means = df.copy()
df_k_means['Cluster'] = labels

# Вывод статистики по кластерам для каждого признака
print("Статистика по кластерам:")
for column in df.columns:
    print(f"\nСтатистика для признака '{column}':")
    cluster_stats = df_k_means.groupby('Cluster')[column].agg(['mean', 'std', 'min', 'max'])
    print(cluster_stats)

# Вычисление коэффициента силуэта
if k > 1:
    silhouette_avg = silhouette_score(df, labels)
    print(f"\nКоэффициент силуэта для k={k}: {silhouette_avg:.4f}")
else:
    print("\nКоэффициент силуэта не определен для k=1")

Статистика по кластерам:

Статистика для признака 'age':
             mean       std       min  max
Cluster                                   
0        0.233909  0.185136  0.000000  1.0
1        0.407482  0.170478  0.013699  1.0
2        0.334299  0.155000  0.000000  1.0

Статистика для признака 'education-num':
             mean       std  min  max
Cluster                              
0        0.587859  0.159466  0.0  1.0
1        0.651716  0.186054  0.0  1.0
2        0.607625  0.176973  0.0  1.0

Статистика для признака 'capital-gain':
             mean       std  min  max
Cluster                              
0        0.003808  0.036722  0.0  1.0
1        0.022471  0.113354  0.0  1.0
2        0.015698  0.088684  0.0  1.0

Статистика для признака 'capital-loss':
             mean       std  min       max
Cluster                                   
0        0.012556  0.073666  0.0  1.000000
1        0.030851  0.114599  0.0  0.648301
2        0.026377  0.104741  0.0  0.648301

Статисти